In [1]:
import pandas as pd
import re
import numpy as np

import requests
import urllib.request
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

In [2]:
import warnings
warnings.filterwarnings('ignore')

### 건강 데이터

In [31]:
# 데이터 프레임 생성
df = pd.DataFrame(np.zeros((180, 6), dtype = np.int), columns=['category', 'title', 'text','url', 'views', 'recommendation'])

In [32]:
df.head(3)

,category,title,text,url,views,recommendation
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0


In [33]:
url = "https://www.kbgoldenlifex.com/senior/XAA72P05010.kb"

# Selenium으로 웹 브라우저를 실행
driver = webdriver.Chrome()
driver.get(url)
page_num = 1
i = 0
while page_num <=9:
    # 웹 페이지가 로드될 때까지 잠시 대기
    driver.implicitly_wait(20) # 15초간 대기

    # 현재 페이지의 HTML을 가져와서 BeautifulSoup으로 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    anchor_tags = soup.find("div", class_ = "list-wrap").find_all("a")
    for tag in anchor_tags:
        href = tag.get('href')
        num = tag.get('data-blts-serno')

        if href and num:
            # 링크를 클릭하여 글 내용 페이지로 이동
            url_in = "https://www.kbgoldenlifex.com/senior/XAA72P05020.kb?bltsDstcd=01&bltsSerno="+str(num)+"&ctntDstcd=01"
            driver.get(url_in)

            # 글 내용이 로드될 때까지 잠시 대기
            driver.implicitly_wait(10) # 10초간 대기

            # 글 내용 페이지의 HTML을 가져와서 BeautifulSoup으로 파싱
            content_html = driver.page_source
            content_soup = BeautifulSoup(content_html, 'html.parser')

            # 글 내용 추
            category = content_soup.find("div", class_ = "view-title").find("span").text
            title = content_soup.find("div", class_ = "view-title").find("h2").text
            views = int(str(content_soup.find("ul", class_ = "conts-util")).split("<span>")[-1].split("</span>")[0].replace(",",""))
            try:
                recommend_url = content_soup.find("div", "news-health-banner").find("a")['href']
            except:
                recommend_url = np.nan
            txt = ""
            tmp = ""

            for con in content_soup.find_all("p"):
                tmp = txt
                txt += con.text.strip() + ""
                if (con.text.strip() == "▶ KB골든라이프X 연관 기사 보기"):
                    txt = tmp
                    recommend_url = con.find("a")['href']
                    break
            txt = ".".join(txt.split(".")[:-1]).replace("\xa0", " ").replace("\n", " ")
            txt = re.sub(r'\s+', ' ', txt)
            df.iloc[i,0] = category
            df.iloc[i,1] = title
            df.iloc[i,2] = txt
            df.iloc[i,3] = url_in
            df.iloc[i,4] = views
            df.iloc[i,5] = recommend_url
            i+=1
            # 이전 페이지로 돌아가기
            driver.back()
    print(page_num, "페이지를 가져왔습니다", "i:", i)
    page_num+=1
    try:
        page_url = 'a[name="pgEvent"][data-page-no="' + str(page_num) + '"]'
        next_page = driver.find_element(By.CSS_SELECTOR, page_url)
        next_page.click()
    except:
        print(page_num, "페이지를 찾을 수 없습니다.")
        break
        
# Selenium 사용이 끝나면 웹 브라우저를 닫음
driver.quit()

1 페이지를 가져왔습니다 i: 20
2 페이지를 가져왔습니다 i: 40
3 페이지를 가져왔습니다 i: 60
4 페이지를 가져왔습니다 i: 80
5 페이지를 가져왔습니다 i: 100
6 페이지를 가져왔습니다 i: 120
7 페이지를 가져왔습니다 i: 140
8 페이지를 가져왔습니다 i: 160
9 페이지를 가져왔습니다 i: 180
10 페이지를 찾을 수 없습니다.


In [34]:
df.head()

,category,title,text,url,views,recommendation
0,건강,알고 먹자 영양제! ③ 비오틴은 정말 탈모에 효과가 있을까?,"‘비타민B7’ ‘비타민H’로도 불리는 비오틴(Biotin)은 모발과 피부, 손톱을 ...",https://www.kbgoldenlifex.com/senior/XAA72P050...,154,http://kbgoldenlifex.com/senior/XAA72P05020.kb...
1,건강,허옇게 일어난 각질이여 안녕~ 여름철 ‘발 미인’ 되는 법,"샌들의 계절, 시선이 머무는 곳은 단연 발뒤꿈치다. 피지선이 없어 다른 부위보다 건...",https://www.kbgoldenlifex.com/senior/XAA72P050...,188,https://kbgoldenlifex.com/senior/XAA72P05020.k...
2,건강,"내 몸의 오랜 통증, 혹시 잘못된 자세 때문?","한 5분 걷거나 서 있었는데, 허리가 뻐근하고 저렸던 경험이 있는가? 갑자기 엉덩이...",https://www.kbgoldenlifex.com/senior/XAA72P050...,166,http://kbgoldenlifex.com/senior/XAA72P05020.kb...
3,건강,나이가 들면 체질도 변한다! 시니어 다이어트 성공비법,"젊었을 때는 많이 먹고 적게 움직여서 살이 쪘지만, 나이가 들면 호르몬 때문에 살이...",https://www.kbgoldenlifex.com/senior/XAA72P050...,357,https://kbgoldenlifex.com/senior/XAA72P05020.k...
4,건강,여름이면 더 심해지는 ‘체취’ 퇴치법,흔히 ‘암내’라고 부르는 체취(몸 냄새)는 여름에 특히 더 심해진다. 보통 타인의 ...,https://www.kbgoldenlifex.com/senior/XAA72P050...,153,http://kbgoldenlifex.com/senior/XAA72P05020.kb...


In [35]:
df.tail()

,category,title,text,url,views,recommendation
175,건강,당신이 남보다 낫다는 착각을 버려라,"자기 자신에 집착하면 자기 자신을 해칠 뿐이다인간은 오랫동안 결점, 실수, 실패, ...",https://www.kbgoldenlifex.com/senior/XAA72P050...,79,NaN
176,건강,숨은 근육 관리하기,척추와 어깨 높이의 관계척추측만은 뒤에서 잘 보인다. 좌우 옆구리 라인이 짝짝이면 ...,https://www.kbgoldenlifex.com/senior/XAA72P050...,576,NaN
177,건강,내 고관절이 어때서,고관절의 나비효과고관절은 골반과 넙다리뼈를 연결하는 중간 관절 부위다. 직립보행을 ...,https://www.kbgoldenlifex.com/senior/XAA72P050...,124,NaN
178,건강,유해물질과 함께 살아가는 법,"전직 식약청 독성부장, 일상 속 유해물질에 대해 말하다양기화 박사는 국내 최초로 ‘...",https://www.kbgoldenlifex.com/senior/XAA72P050...,361,NaN
179,건강,목과 손의 노화를 잡는 5가지 방법,TIP 1 손바닥으로 하늘을 가리지 말 것손의 노화를 부르는 대표적인 요인은 손으로...,https://www.kbgoldenlifex.com/senior/XAA72P050...,450,NaN


In [36]:
df.to_csv('KB골든라이프X_건강.csv', index = False, encoding = 'utf-8')

### 여가 데이터

In [3]:
# 데이터 프레임 생성
df = pd.DataFrame(np.zeros((183, 6), dtype = np.int), columns=['category', 'title', 'text','url', 'views', 'recommendation'])

In [4]:
df.head()

,category,title,text,url,views,recommendation
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


In [5]:
url = "https://www.kbgoldenlifex.com/senior/XAA72P05011.kb"

# Selenium으로 웹 브라우저를 실행
driver = webdriver.Chrome()
driver.get(url)
page_num = 1
i = 0
while page_num <=10:
    # 웹 페이지가 로드될 때까지 잠시 대기
    driver.implicitly_wait(23) # 23초간 대기

    # 현재 페이지의 HTML을 가져와서 BeautifulSoup으로 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    anchor_tags = soup.find("div", class_ = "list-wrap").find_all("a")
    for tag in anchor_tags:
        href = tag.get('href')
        num = tag.get('data-blts-serno')

        if href and num:
            # 링크를 클릭하여 글 내용 페이지로 이동
            url_in = "https://www.kbgoldenlifex.com/senior/XAA72P05020.kb?bltsDstcd=01&bltsSerno="+str(num)+"&ctntDstcd=01"
            driver.get(url_in)

            # 글 내용이 로드될 때까지 잠시 대기
            driver.implicitly_wait(13) # 10초간 대기

            # 글 내용 페이지의 HTML을 가져와서 BeautifulSoup으로 파싱
            content_html = driver.page_source
            content_soup = BeautifulSoup(content_html, 'html.parser')

            # 글 내용 추
            category = content_soup.find("div", class_ = "view-title").find("span").text
            title = content_soup.find("div", class_ = "view-title").find("h2").text
            views = int(str(content_soup.find("ul", class_ = "conts-util")).split("<span>")[-1].split("</span>")[0].replace(",",""))
            try:
                recommend_url = content_soup.find("div", "news-health-banner").find("a")['href']
            except:
                recommend_url = np.nan
            txt = ""
            tmp = ""

            for con in content_soup.find_all("p"):
                tmp = txt
                txt += con.text.strip() + ""
                if (con.text.strip() == "▶ KB골든라이프X 연관 기사 보기"):
                    txt = tmp
                    try:
                        recommend_url = con.find("a")['href']
                    except:
                        recommend_url = recommend_url
                    break
            txt = ".".join(txt.split(".")[:-1]).replace("\xa0", " ").replace("\n", " ")
            txt = re.sub(r'\s+', ' ', txt)
            df.iloc[i,0] = category
            df.iloc[i,1] = title
            df.iloc[i,2] = txt
            df.iloc[i,3] = url_in
            df.iloc[i,4] = views
            df.iloc[i,5] = recommend_url
            i+=1
            # 이전 페이지로 돌아가기
            driver.back()
    print(page_num, "페이지를 가져왔습니다", "i:", i)
    page_num+=1
    try:
        page_url = 'a[name="pgEvent"][data-page-no="' + str(page_num) + '"]'
        next_page = driver.find_element(By.CSS_SELECTOR, page_url)
        next_page.click()
    except:
        print(page_num, "페이지를 찾을 수 없습니다.")
        break
        
# Selenium 사용이 끝나면 웹 브라우저를 닫음
driver.quit()

1 페이지를 가져왔습니다 i: 20
2 페이지를 가져왔습니다 i: 40
3 페이지를 가져왔습니다 i: 60
4 페이지를 가져왔습니다 i: 80
5 페이지를 가져왔습니다 i: 100
6 페이지를 가져왔습니다 i: 120
7 페이지를 가져왔습니다 i: 140
8 페이지를 가져왔습니다 i: 160
9 페이지를 가져왔습니다 i: 180
10 페이지를 가져왔습니다 i: 183
11 페이지를 찾을 수 없습니다.


In [6]:
df.head(3)

,category,title,text,url,views,recommendation
0,여가,"키오스크, 두렵지 않아! 즐기며 배우는 키오스크 활용법",최저임금이 크게 뛰기 시작한 2018년을 기점으로 주목받기 시작한 키오스크(공공장소...,https://www.kbgoldenlifex.com/senior/XAA72P050...,45,https://kbgoldenlifex.com/senior/XAA72P05020.k...
1,여가,"싱그러운 여름꽃, 네 이름은 뭐니? 식물 찾기 만능 앱","산에 핀 야생화나 화단에 핀 꽃의 이름이 궁금한데, 알 방법이 없어 답답했던 적이 ...",https://www.kbgoldenlifex.com/senior/XAA72P050...,87,https://kbgoldenlifex.com/senior/XAA72P05020.k...
2,여가,구석구석 로컬 투어 ③ ‘낮밤’ 없이 즐거운 광양 여행,‘구석구석 로컬 투어’ 시리즈는 국내 구석구석 로컬 여행지의 멋과 맛을 큐레이션합니...,https://www.kbgoldenlifex.com/senior/XAA72P050...,215,https://kbgoldenlifex.com/senior/XAA72P05020.k...


In [7]:
df.tail()

,category,title,text,url,views,recommendation
178,여가,서울에서 느끼는 바르셀로나 본토의 맛,"미식의 도시, 바스크의 핀초스바스크는 스페인 북부의 소수 언어 지역이다. 바스크 지...",https://www.kbgoldenlifex.com/senior/XAA72P050...,661,NaN
179,여가,코로나로부터 자유로운 여행,북극 다큐멘터리 속으로LINDBLAD EXPEDITIONS린드블라드 익스페디션은 북...,https://www.kbgoldenlifex.com/senior/XAA72P050...,439,NaN
180,여가,코로나로부터 자유로운 여행,북극 다큐멘터리 속으로LINDBLAD EXPEDITIONS린드블라드 익스페디션은 북...,https://www.kbgoldenlifex.com/senior/XAA72P050...,440,NaN
181,여가,레드도 화이트도 아닌 내추럴 와인,내추럴 와인이 뭐길래?내추럴 와인은 무엇인가? 그럼 우리가 마셨던 와인은 내추럴하지...,https://www.kbgoldenlifex.com/senior/XAA72P050...,40,NaN
182,여가,나 혼자 한 잔씩 마실 때 이런 위스키,위스키 한 잔은 몸과 마음을 녹여주는 것 같다. 한 잔씩 마시는 위스키는 110세까...,https://www.kbgoldenlifex.com/senior/XAA72P050...,604,NaN


In [8]:
df.to_csv('KB골든라이프X_여가.csv', index = False, encoding = 'utf-8')

### 트렌드 데이터

In [9]:
# 데이터 프레임 생성
df = pd.DataFrame(np.zeros((144, 6), dtype = np.int), columns=['category', 'title', 'text','url', 'views', 'recommendation'])

In [10]:
df.head()

,category,title,text,url,views,recommendation
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


In [11]:
url = "https://www.kbgoldenlifex.com/senior/XAA72P05012.kb"

# Selenium으로 웹 브라우저를 실행
driver = webdriver.Chrome()
driver.get(url)
page_num = 1
i = 0
while page_num <=8:
    # 웹 페이지가 로드될 때까지 잠시 대기
    driver.implicitly_wait(23) # 23초간 대기

    # 현재 페이지의 HTML을 가져와서 BeautifulSoup으로 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    anchor_tags = soup.find("div", class_ = "list-wrap").find_all("a")
    for tag in anchor_tags:
        href = tag.get('href')
        num = tag.get('data-blts-serno')

        if href and num:
            # 링크를 클릭하여 글 내용 페이지로 이동
            url_in = "https://www.kbgoldenlifex.com/senior/XAA72P05020.kb?bltsDstcd=01&bltsSerno="+str(num)+"&ctntDstcd=01"
            driver.get(url_in)

            # 글 내용이 로드될 때까지 잠시 대기
            driver.implicitly_wait(13) # 10초간 대기

            # 글 내용 페이지의 HTML을 가져와서 BeautifulSoup으로 파싱
            content_html = driver.page_source
            content_soup = BeautifulSoup(content_html, 'html.parser')

            # 글 내용 추
            category = content_soup.find("div", class_ = "view-title").find("span").text
            title = content_soup.find("div", class_ = "view-title").find("h2").text
            views = int(str(content_soup.find("ul", class_ = "conts-util")).split("<span>")[-1].split("</span>")[0].replace(",",""))
            try:
                recommend_url = content_soup.find("div", "news-health-banner").find("a")['href']
            except:
                recommend_url = np.nan
            txt = ""
            tmp = ""

            for con in content_soup.find_all("p"):
                tmp = txt
                txt += con.text.strip() + ""
                if (con.text.strip() == "▶ KB골든라이프X 연관 기사 보기"):
                    txt = tmp
                    try:
                        recommend_url = con.find("a")['href']
                    except:
                        recommend_url = recommend_url
                    break
            txt = ".".join(txt.split(".")[:-1]).replace("\xa0", " ").replace("\n", " ")
            txt = re.sub(r'\s+', ' ', txt)
            df.iloc[i,0] = category
            df.iloc[i,1] = title
            df.iloc[i,2] = txt
            df.iloc[i,3] = url_in
            df.iloc[i,4] = views
            df.iloc[i,5] = recommend_url
            i+=1
            # 이전 페이지로 돌아가기
            driver.back()
    print(page_num, "페이지를 가져왔습니다", "i:", i)
    page_num+=1
    try:
        page_url = 'a[name="pgEvent"][data-page-no="' + str(page_num) + '"]'
        next_page = driver.find_element(By.CSS_SELECTOR, page_url)
        next_page.click()
    except:
        print(page_num, "페이지를 찾을 수 없습니다.")
        break
        
# Selenium 사용이 끝나면 웹 브라우저를 닫음
driver.quit()

1 페이지를 가져왔습니다 i: 20
2 페이지를 가져왔습니다 i: 40
3 페이지를 가져왔습니다 i: 60
4 페이지를 가져왔습니다 i: 80
5 페이지를 가져왔습니다 i: 100
6 페이지를 가져왔습니다 i: 120
7 페이지를 가져왔습니다 i: 140
8 페이지를 가져왔습니다 i: 144
9 페이지를 찾을 수 없습니다.


In [12]:
df.head(23)

,category,title,text,url,views,recommendation
0,트렌드,"New 직업 시리즈 ⑨ 취미도 즐기고, 돈도 벌고! 시니어 바리스타",은퇴 후 직업을 찾을 때 가장 먼저 고려해야 할 것 중 하나는 ‘취미’다. 좋아서 ...,https://www.kbgoldenlifex.com/senior/XAA72P050...,49,http://kbgoldenlifex.com/senior/XAA72P05020.kb...
1,트렌드,"가성비부터 럭셔리까지, 실버타운, 어디가 좋아요?","‘세 끼 식사 제공, 의료진과 운동 매니저 상주, 부속 한의원, 주 2회 청소, 수...",https://www.kbgoldenlifex.com/senior/XAA72P050...,408,https://kbgoldenlifex.com/senior/XAA72P05020.k...
2,트렌드,"전기세가 폭탄이 되지 않게, 걱정 없이 여름 나는 절전법",전기 소비량이 연중 최대가 되는 한여름이 돌아왔다. 더불어 전기요금 인상 소식도 시...,https://www.kbgoldenlifex.com/senior/XAA72P050...,1832,https://kbgoldenlifex.com/senior/XAA72P05020.k...
3,트렌드,내가 전생에 왕이었다면 어떤 왕이었을까? 킹BTI 테스트,,https://www.kbgoldenlifex.com/senior/XAA72P050...,111309,NaN
4,트렌드,이게 무슨 말이래? 요즘 애들 신조어 사전,"군싹, 알잘딱깔센? 분명 우리나라 말인데, 도통 무슨 말인지 알 수 없는 1020세...",https://www.kbgoldenlifex.com/senior/XAA72P050...,223,http://kbgoldenlifex.com/senior/XAA72P05020.kb...
5,트렌드,"환경도 보호하고, 돈도 번다! 떠오르는 요즘 재테크, 쓰테크!",쓰레기로 돈을 버는 ‘쓰테크(쓰레기와 재테크의 합성어)’가 인기다. 폐지나 공병 얘...,https://www.kbgoldenlifex.com/senior/XAA72P050...,360,https://www.kbgoldenlifex.com/senior/XAA72P050...
6,트렌드,New 직업 시리즈 ⑧ 실버인지놀이지도사,고령화가 가속화되면서 노인 케어를 위한 직업이 주목받고 있다. ‘실버인지놀이지도사’...,https://www.kbgoldenlifex.com/senior/XAA72P050...,295,https://www.kbgoldenlifex.com/senior/XAA72P050...
7,트렌드,New 직업 시리즈 ⑦일 자체가 힐링! 숲해설가,일상의 고단함을 다독여주는 공간이자 면역력까지 높인다는 숲. 코로나19 이후 지친 ...,https://www.kbgoldenlifex.com/senior/XAA72P050...,1692,NaN
8,트렌드,도심 속 산책이 필요한 당신에게,빠르게 돌아가는 서울 한복판에서도 느긋함과 기다림의 미학으로 손짓하는 휴식 공간을 ...,https://www.kbgoldenlifex.com/senior/XAA72P050...,141,http://kbgoldenlifex.com/senior/XAA72P05020.kb...
9,트렌드,‘K-빨간 맛’의 질주,"‘불닭볶음면’, ‘엽기떡볶이’, ‘신길동 매운 짬뽕’, ‘실비 김치’ 등등. 이름만...",https://www.kbgoldenlifex.com/senior/XAA72P050...,68,NaN


In [13]:
df.tail()

,category,title,text,url,views,recommendation
139,트렌드,여전히 멋진 호텔들이 있다,브라슈 호텔브라슈 호텔은 프랑스 호텔 그룹 이보크가 세계적 산업디자이너 필립 스탁과...,https://www.kbgoldenlifex.com/senior/XAA72P050...,113,NaN
140,트렌드,핸드크림 뭐 쓰세요?,얼굴보다 손 관리를 더 철저하게노화는 얼굴뿐 아니라 우리 몸 전체에서 동시다발적으로...,https://www.kbgoldenlifex.com/senior/XAA72P050...,833,NaN
141,트렌드,유산균은 피부에도 좋을까?,남들은 좋다는 화장품인데 왜 내가 바르면 뾰루지가 날까?어느 브랜드의 발효 성분 에...,https://www.kbgoldenlifex.com/senior/XAA72P050...,30,NaN
142,트렌드,갖지 않고 즐기는 삶,"굳이 돈을 주고 사서 내 것으로 만들지 않아도, 소유하는 것만큼 즐거울 수 있다.요...",https://www.kbgoldenlifex.com/senior/XAA72P050...,53,NaN
143,트렌드,서점이 달라지고 있다,서점은 여전하다말끔한 공간에 감각적인 표지의 책들이 놓여 있다. 건물 지하 공간에 ...,https://www.kbgoldenlifex.com/senior/XAA72P050...,546,NaN


In [17]:
df.recommendation.isna().sum()

48

In [18]:
df.to_csv('KB골든라이프X_트렌드.csv', index = False, encoding = 'utf-8')

## 연금/은퇴 데이터

In [19]:
# 데이터 프레임 생성
df = pd.DataFrame(np.zeros((136, 6), dtype = np.int), columns=['category', 'title', 'text','url', 'views', 'recommendation'])

In [20]:
df.head()

,category,title,text,url,views,recommendation
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


In [21]:
url = "https://www.kbgoldenlifex.com/senior/XAA72P06011.kb"

# Selenium으로 웹 브라우저를 실행
driver = webdriver.Chrome()
driver.get(url)
page_num = 1
i = 0
while page_num <=14:
    # 웹 페이지가 로드될 때까지 잠시 대기
    driver.implicitly_wait(23) # 23초간 대기

    # 현재 페이지의 HTML을 가져와서 BeautifulSoup으로 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    anchor_tags = soup.find("div", class_ = "list-wrap").find_all("a")
    for tag in anchor_tags:
        href = tag.get('href')
        num = tag.get('data-blts-serno')

        if href and num:
            # 링크를 클릭하여 글 내용 페이지로 이동
            url_in = "https://www.kbgoldenlifex.com/senior/XAA72P06020.kb?bltsDstcd=01&bltsSerno="+str(num)+"&ctntDstcd=01"
            driver.get(url_in)

            # 글 내용이 로드될 때까지 잠시 대기
            driver.implicitly_wait(13) # 10초간 대기

            # 글 내용 페이지의 HTML을 가져와서 BeautifulSoup으로 파싱
            content_html = driver.page_source
            content_soup = BeautifulSoup(content_html, 'html.parser')

            # 글 내용 추
            category = content_soup.find("div", class_ = "view-title").find("span").text
            title = content_soup.find("div", class_ = "view-title").find("h2").text
            views = int(str(content_soup.find("ul", class_ = "conts-util")).split("<span>")[-1].split("</span>")[0].replace(",",""))
            try:
                recommend_url = content_soup.find("div", "news-health-banner").find("a")['href']
            except:
                recommend_url = np.nan
            txt = ""
            tmp = ""

            for con in content_soup.find_all("p"):
                tmp = txt
                txt += con.text.strip() + ""
                if (con.text.strip() == "▶ KB골든라이프X 연관 기사 보기"):
                    txt = tmp
                    try:
                        recommend_url = con.find("a")['href']
                    except:
                        recommend_url = recommend_url
                    break
            txt = ".".join(txt.split(".")[:-1]).replace("\xa0", " ").replace("\n", " ")
            txt = re.sub(r'\s+', ' ', txt)
            df.iloc[i,0] = category
            df.iloc[i,1] = title
            df.iloc[i,2] = txt
            df.iloc[i,3] = url_in
            df.iloc[i,4] = views
            df.iloc[i,5] = recommend_url
            i+=1
            # 이전 페이지로 돌아가기
            driver.back()
    print(page_num, "페이지를 가져왔습니다", "i:", i)
    page_num+=1
    try:
        page_url = 'a[name="pgEvent"][data-page-no="' + str(page_num) + '"]'
        next_page = driver.find_element(By.CSS_SELECTOR, page_url)
        next_page.click()
    except:
        print(page_num, "페이지를 찾을 수 없습니다.")
        break
        
# Selenium 사용이 끝나면 웹 브라우저를 닫음
driver.quit()

1 페이지를 가져왔습니다 i: 10
2 페이지를 가져왔습니다 i: 20
3 페이지를 가져왔습니다 i: 30
4 페이지를 가져왔습니다 i: 40
5 페이지를 가져왔습니다 i: 50
6 페이지를 가져왔습니다 i: 60
7 페이지를 가져왔습니다 i: 70
8 페이지를 가져왔습니다 i: 80
9 페이지를 가져왔습니다 i: 90
10 페이지를 가져왔습니다 i: 100
11 페이지를 가져왔습니다 i: 110
12 페이지를 가져왔습니다 i: 120
13 페이지를 가져왔습니다 i: 130
14 페이지를 가져왔습니다 i: 136
15 페이지를 찾을 수 없습니다.


In [23]:
df.head()

,category,title,text,url,views,recommendation
0,연금/은퇴,[KB골든라이프센터] 퇴직 후 건강보험료 줄이는 7가지 방법,오늘 주제 한눈에 보기 오랜 회사생활을 마치고 은퇴를 앞둔 직장인에게 은퇴 후 소득...,https://www.kbgoldenlifex.com/senior/XAA72P060...,145,https://www.kbgoldenlifex.com/senior/XAA72P060...
1,연금/은퇴,[KB골든라이프센터] 은퇴 후 생활비 350만원 만들기,오늘 주제 한눈에 보기 소득이 크게 줄어드는 은퇴 이후의 삶. 이를 은퇴 이전처럼 ...,https://www.kbgoldenlifex.com/senior/XAA72P060...,527,https://www.kbgoldenlifex.com/senior/XAA72P060...
2,연금/은퇴,[KB골든라이프센터] 기초연금이 뭐에요? 제가 신청해야 받을 수 있나요?,Q1. 기초연금이 뭐예요? 어떤 사람들이 받나요?기초연금은 만 65세 이상 국내에 ...,https://www.kbgoldenlifex.com/senior/XAA72P060...,2437,https://www.kbgoldenlifex.com/senior/XAA72P060...
3,연금/은퇴,"내 금융소득과 건강보험료, 따져 보셨나요?",오늘 주제 한눈에 보기 퇴직을 앞두고 가장 먼저 준비해야 하는 것은 재무적인 항목이...,https://www.kbgoldenlifex.com/senior/XAA72P060...,270,https://www.kbgoldenlifex.com/senior/XAA72P060...
4,연금/은퇴,[KB골든라이프센터 ] ‘1인가구 전성시대’ 행복한 은퇴준비 노하우,오늘 주제 한눈에 보기 얼마전 지인 셋과 모처럼 북한산 둘레길을 걸었다. 셋은 모두...,https://www.kbgoldenlifex.com/senior/XAA72P060...,183,https://www.kbgoldenlifex.com/senior/XAA72P060...


In [24]:
df.tail()

,category,title,text,url,views,recommendation
131,연금/은퇴,"핫한 공모주 시장, 펀드를 활용한 간접투자 포인트 TIP","2020년 하반기, 초저금리로 인해 풍부해진 유동성과 대박에 대한 기대감이 맞물리면...",https://www.kbgoldenlifex.com/senior/XAA72P060...,67,NaN
132,연금/은퇴,정부의 새 경제 정책 알아보기! 금융 세제 선진화 추진 방향 주요 내용,"2020년 6월 25일, 기획재정부가 금융 세제 선진화 추진 방향을 발표했으며, 2...",https://www.kbgoldenlifex.com/senior/XAA72P060...,22,NaN
133,연금/은퇴,베이비부머 노후대비 위한 자산관리 포인트,은퇴한 베이비부머들은 재무적으로 두 가지 리스크에 노출돼 있다. 하나는 저금리 기조...,https://www.kbgoldenlifex.com/senior/XAA72P060...,42,NaN
134,연금/은퇴,상위 0.68% 부자들의 돈버는 전략 1순위는? 주식VS부동산,안녕하세요. 허생원(許生員)입니다.(전편)에서 ‘한국의 백만장자들은 부동산에 얼마나...,https://www.kbgoldenlifex.com/senior/XAA72P060...,35,NaN
135,연금/은퇴,"2010년대는 미국 기술주, 2020년대의 성장주는?",팬데믹 이후 전 세계 증시를 이끌던 미국 대형 기술주의 상승세가 주춤하고 있다는 사...,https://www.kbgoldenlifex.com/senior/XAA72P060...,27,NaN


In [25]:
# recommendation 없는 행들
df.recommendation.isna().sum()

55

In [26]:
df.to_csv('KB골든라이프X_연금_은퇴.csv', index = False, encoding = 'utf-8')

## 부동산 데이터

In [20]:
# 데이터 프레임 생성
df = pd.DataFrame(np.zeros((181, 6), dtype = np.int), columns=['category', 'title', 'text','url', 'views', 'recommendation'])

In [21]:
df.head()

,category,title,text,url,views,recommendation
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


In [24]:
url = "https://www.kbgoldenlifex.com/senior/XAA72P06010.kb"

# Selenium으로 웹 브라우저를 실행
driver = webdriver.Chrome()
driver.get(url)
page_num = 1
i = 0
while page_num <=19:
    # 웹 페이지가 로드될 때까지 잠시 대기
    driver.implicitly_wait(27) # 23초간 대기

    # 현재 페이지의 HTML을 가져와서 BeautifulSoup으로 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    anchor_tags = soup.find("div", class_ = "list-wrap").find_all("a")
    for tag in anchor_tags:
        href = tag.get('href')
        num = tag.get('data-blts-serno')

        if href and num:
            # 링크를 클릭하여 글 내용 페이지로 이동
            url_in = "https://www.kbgoldenlifex.com/senior/XAA72P06020.kb?bltsDstcd=01&bltsSerno="+str(num)+"&ctntDstcd=01"
            driver.get(url_in)

            # 글 내용이 로드될 때까지 잠시 대기
            driver.implicitly_wait(13) # 10초간 대기

            # 글 내용 페이지의 HTML을 가져와서 BeautifulSoup으로 파싱
            content_html = driver.page_source
            content_soup = BeautifulSoup(content_html, 'html.parser')

            # 글 내용 추
            category = content_soup.find("div", class_ = "view-title").find("span").text
            title = content_soup.find("div", class_ = "view-title").find("h2").text
            views = int(str(content_soup.find("ul", class_ = "conts-util")).split("<span>")[-1].split("</span>")[0].replace(",",""))
            try:
                recommend_url = content_soup.find("div", "news-health-banner").find("a")['href']
            except:
                recommend_url = np.nan
            txt = ""
            tmp = ""

            for con in content_soup.find_all("p"):
                tmp = txt
                txt += con.text.strip() + ""
                if (con.text.strip() == "▶ KB골든라이프X 연관 기사 보기"):
                    txt = tmp
                    try:
                        recommend_url = con.find("a")['href']
                    except:
                        recommend_url = recommend_url
                    break
            txt = ".".join(txt.split(".")[:-1]).replace("\xa0", " ").replace("\n", " ")
            txt = re.sub(r'\s+', ' ', txt)
            df.iloc[i,0] = category
            df.iloc[i,1] = title
            df.iloc[i,2] = txt
            df.iloc[i,3] = url_in
            df.iloc[i,4] = views
            df.iloc[i,5] = recommend_url
            i+=1
            # 이전 페이지로 돌아가기
            driver.back()
    print(page_num, "페이지를 가져왔습니다", "i:", i)
    page_num+=1
    try:
        page_url = 'a[name="pgEvent"][data-page-no="' + str(page_num) + '"]'
        next_page = driver.find_element(By.CSS_SELECTOR, page_url)
        next_page.click()
    except:
        print(page_num, "페이지를 찾을 수 없습니다.")
        break
        
# Selenium 사용이 끝나면 웹 브라우저를 닫음
driver.quit()

1 페이지를 가져왔습니다 i: 10
2 페이지를 가져왔습니다 i: 20
3 페이지를 가져왔습니다 i: 30
4 페이지를 가져왔습니다 i: 40
5 페이지를 가져왔습니다 i: 50
6 페이지를 가져왔습니다 i: 60
7 페이지를 가져왔습니다 i: 70
8 페이지를 가져왔습니다 i: 80
9 페이지를 가져왔습니다 i: 90
10 페이지를 가져왔습니다 i: 100
11 페이지를 가져왔습니다 i: 110
12 페이지를 가져왔습니다 i: 120
13 페이지를 가져왔습니다 i: 130
14 페이지를 가져왔습니다 i: 140
15 페이지를 가져왔습니다 i: 150
16 페이지를 가져왔습니다 i: 160
17 페이지를 가져왔습니다 i: 170
18 페이지를 가져왔습니다 i: 180
19 페이지를 가져왔습니다 i: 181
20 페이지를 찾을 수 없습니다.


In [25]:
df.head()

,category,title,text,url,views,recommendation
0,부동산,"경기침체 우려된다는데, 부동산의 미래는?",오늘 주제 한눈에 보기 부동산 투자 시 눈여겨보는 지표가 기준금리입니다. 더불어 기...,https://www.kbgoldenlifex.com/senior/XAA72P060...,26,https://www.kbgoldenlifex.com/senior/XAA72P060...
1,부동산,“모 아니면 도” 갈수록 심해지는 서울ㆍ지방 청약시장 양극화,오늘 주제 한눈에 보기 올해 상반기 아파트 청약시장은 한마디로 ‘지역별 양극화 심화...,https://www.kbgoldenlifex.com/senior/XAA72P060...,41,https://www.kbgoldenlifex.com/senior/XAA72P060...
2,부동산,“보증금 깎아줄게요” 전월세 갱신계약 줄었지만 감액계약 비중 역대 최대,오늘 주제 한눈에 보기 서울 아파트 시세 하락이 안정을 찾고 있습니다. 다만 강남권...,https://www.kbgoldenlifex.com/senior/XAA72P060...,21,https://www.kbgoldenlifex.com/senior/XAA72P060...
3,부동산,"하락세 강했던 올 상반기, 아파트값 가장 많이 빠진 곳은?",오늘 주제 한눈에 보기 어느새 2023년 상반기가 끝이 났습니다. 올해 상반기 아파...,https://www.kbgoldenlifex.com/senior/XAA72P060...,26,https://www.kbgoldenlifex.com/senior/XAA72P060...
4,부동산,"은퇴를 앞둔 부부, 아파텔로 갈까요? 말까요?",오늘 주제 한눈에 보기 최근 몇 년 새 아파트 시장 붐을 타고 '아파텔'이라는 이름...,https://www.kbgoldenlifex.com/senior/XAA72P060...,253,https://www.kbgoldenlifex.com/senior/XAA72P060...


In [26]:
df.tail()

,category,title,text,url,views,recommendation
176,부동산,"""한번 더 가능해요?” 전·월세 계약, 묵시적 갱신 vs 갱신요구",주택임대차보호법이 최근 개정돼 임차인이 적극적으로 임대차계약을 갱신해 줄 것을 요구...,https://www.kbgoldenlifex.com/senior/XAA72P060...,261,NaN
177,부동산,"전세, 과연 사라질까요?",임대차 3법이 시행되자 전세 시장이 흔들리고 있습니다. 부동산 시장에서는 전세 매물...,https://www.kbgoldenlifex.com/senior/XAA72P060...,27,NaN
178,부동산,1주택자로 잘 사는 법,오늘은 좀 더 실용적인 얘기를 해보자.집 한 채 달랑 갖고 있는 사람들을 위한 자산...,https://www.kbgoldenlifex.com/senior/XAA72P060...,50,NaN
179,부동산,집 짓기 전에 남의 집 ‘현판’을 먼저 살펴야 하는 이유,# 건축주와 건축업체 교감이 중요위와 같은 방법으로 현판에 있는 업체를 몇 군데 인...,https://www.kbgoldenlifex.com/senior/XAA72P060...,90,NaN
180,부동산,"리모델링, 재건축의 대안이 될 수 있을까?",우리나라는 70~80년대 도시화 과정을 거치면서 단기간에 고밀도 개발이 진행됐다. ...,https://www.kbgoldenlifex.com/senior/XAA72P060...,78,NaN


In [27]:
# recommendation 없는 행들
df[~df.recommendation.isna()].shape

(135, 6)

In [28]:
df.to_csv('KB골든라이프X_부동산.csv', index = False, encoding = 'utf-8')

## 데이터 병합

In [7]:
df_건강 = pd.read_csv("KB골든라이프X_건강.csv", encoding = 'cp949')
df_부동산 = pd.read_csv("KB골든라이프X_부동산.csv", encoding = 'cp949')
df_여가 = pd.read_csv("KB골든라이프X_여가.csv", encoding = 'cp949')
df_연금_은퇴 = pd.read_csv("KB골든라이프X_연금_은퇴.csv", encoding = 'cp949')
df_트렌드 = pd.read_csv("KB골든라이프X_트렌드.csv", encoding = 'cp949')

In [20]:
df_통합 = pd.concat([df_건강, df_부동산, df_여가, df_연금_은퇴, df_트렌드])
df_통합.rename(columns={'views':'hit'}, inplace = True)
df_통합.reset_index(drop = True, inplace = True)

In [21]:
df_통합.head()

,category,title,text,url,hit,recommendation
0,건강,알고 먹자 영양제! ③ 비오틴은 정말 탈모에 효과가 있을까?,"‘비타민B7’ ‘비타민H’로도 불리는 비오틴(Biotin)은 모발과 피부, 손톱을 ...",https://www.kbgoldenlifex.com/senior/XAA72P050...,154,http://kbgoldenlifex.com/senior/XAA72P05020.kb...
1,건강,허옇게 일어난 각질이여 안녕~ 여름철 ‘발 미인’ 되는 법,"샌들의 계절, 시선이 머무는 곳은 단연 발뒤꿈치다. 피지선이 없어 다른 부위보다 건...",https://www.kbgoldenlifex.com/senior/XAA72P050...,188,https://kbgoldenlifex.com/senior/XAA72P05020.k...
2,건강,"내 몸의 오랜 통증, 혹시 잘못된 자세 때문?","한 5분 걷거나 서 있었는데, 허리가 뻐근하고 저렸던 경험이 있는가? 갑자기 엉덩이...",https://www.kbgoldenlifex.com/senior/XAA72P050...,166,http://kbgoldenlifex.com/senior/XAA72P05020.kb...
3,건강,나이가 들면 체질도 변한다! 시니어 다이어트 성공비법,"젊었을 때는 많이 먹고 적게 움직여서 살이 쪘지만, 나이가 들면 호르몬 때문에 살이...",https://www.kbgoldenlifex.com/senior/XAA72P050...,357,https://kbgoldenlifex.com/senior/XAA72P05020.k...
4,건강,여름이면 더 심해지는 ‘체취’ 퇴치법,흔히 ‘암내’라고 부르는 체취(몸 냄새)는 여름에 특히 더 심해진다. 보통 타인의 ...,https://www.kbgoldenlifex.com/senior/XAA72P050...,153,http://kbgoldenlifex.com/senior/XAA72P05020.kb...


In [17]:
df_통합.shape

(824, 6)

In [24]:
df_통합.to_csv("KB골든라이프X_통합.csv", index = False, encoding = 'utf-8')